In [33]:
import numpy as np
import torch as T
import torch.nn as nn
import torch.nn.functional as F

T.manual_seed(10)

# in_var = T.randn(2,requires_grad=True)
# in_var = T.tensor([[0.5,-0.6],[0.7,1.],[0.2,0.2]],requires_grad=True)
in_var = T.tensor([[1.,2.],[0.,-1.]],requires_grad=True)

NN1 = nn.Linear(2,1)
nn.init.uniform_(NN1.weight.data,-1,1)
nn.init.constant_(NN1.weight.data,1)
nn.init.constant_(NN1.bias.data,-1)

NN2 = nn.Linear(2,1)
NN2.load_state_dict(dict(NN1.named_parameters()))
print(NN1.weight.data)
print(NN2.weight.data)


out_var = NN1(in_var)

# print(out_var)
pointer_param_NN1  = NN1.named_parameters()
print(dict(pointer_param_NN1))

check_effect_detach = 0
if check_effect_detach == 1:
    # sau khi detach, thong tin ve grad bien mat
    out_var = out_var.detach()
    out_var = out_var.numpy()
    lost_1 = out_var.sum()/out_var.shape[0]
    lost_1 = T.tensor(lost_1,requires_grad=True)
    print(lost_1)
    lost_1.backward()
    print(NN1.weight.grad)
elif check_effect_detach == 2:
    # khong detach, thong tin grad van co n
    lost = T.mean(out_var)
    print(lost)
    lost.backward()
    print(NN1.weight.grad)

print(in_var.shape[0])
if in_var.shape[0]>1:
    BN1 = nn.BatchNorm1d(1)
    out_var_bn = BN1(out_var)
    print(out_var_bn)
    out_var_bn = T.mean(out_var_bn)
    print(out_var_bn)
    out_var_bn.backward()
    print(NN1.weight.grad)

tensor([[1., 1.]])
tensor([[1., 1.]])
{'weight': Parameter containing:
tensor([[1., 1.]], requires_grad=True), 'bias': Parameter containing:
tensor([-1.], requires_grad=True)}
2
tensor([[ 1.0000],
        [-1.0000]], grad_fn=<NativeBatchNormBackward>)
tensor(0., grad_fn=<MeanBackward0>)
tensor([[0., 0.]])
